関数の部分は変更する点はない。パス指定と指定のCSVをダウンロードしておいてほしい。


In [1]:
from arelle import ModelManager
from arelle import Cntlr
import os
import zipfile
import glob
import pandas as pd
import datetime
import requests

In [7]:
def make_edinet_info_list(edinetcodedlinfo_filepath):
    edinet_info = pd.read_csv(edinetcodedlinfo_filepath, skiprows=1,
                                 encoding='cp932')
    edinet_info = edinet_info[["ＥＤＩＮＥＴコード","提出者業種"]]
    edinet_info_list = edinet_info.values.tolist()
    return edinet_info_list

In [10]:
def unzip_file(zip_dir,xbrl_file_expressions):
    zip_files = glob.glob(os.path.join(zip_dir, '*.zip'))

    number_of_zip_lists = len(zip_files)
    print("number_of_zip_lists：", number_of_zip_lists)

    for index, zip_file in enumerate(zip_files):
        print(zip_file, ":", index + 1, "/", number_of_zip_lists)
        with zipfile.ZipFile(zip_file) as zip_f:
            zip_f.extractall(zip_dir)
            zip_f.close()

    xbrl_files = glob.glob(xbrl_file_expressions)
    return xbrl_files

In [1]:
def make_edinet_company_info_list(xbrl_files,edinet_info_list):
    edinet_company_info_list = []
    minimum_list = []
    for index, xbrl_file in enumerate(xbrl_files):
        
        edinet_code = ""  # EDINETCODE
        security_code = '' ##証券コード
        filer_name_jp = ''  # 企業名
        industry_code = []  # 業種
        
        price_earningS_ratios = [] ##PER
        net_asset_per_shares = [] #BPS
        basic_earnings_per_shares = [] #EPS
        diluted_earnings_per_shares = [] #調整後EPS
        dividend_per_shares = [] ##DPS
        return_on_equitys = [] ##ROE
        dividend_payout_ratios = [] ##配当性向
        cash_flow_from_mains = [] ##営業活動によるキャッシュフロー
        cash_flow_from_investings = [] ##投資活動によるキャッシュフロー
        net_incomes = [] ##当期純利益
        
        
        
        company_info_list = []  # 企業情報

        ctrl = Cntlr.Cntlr()
        model_manager = ModelManager.initialize(ctrl)
        model_xbrl = model_manager.load(xbrl_file)

        print(xbrl_file, ":", index + 1, "/", len(xbrl_files))

        for fact in model_xbrl.facts:

            if fact.concept.qname.localName == 'EDINETCodeDEI':
                print("EDINETコード", fact.value)
                edinet_code = fact.value

                for code_name in edinet_info_list:
                    if code_name[0] == edinet_code:
                        print("業種",code_name[1])
                        industry_code = code_name[1]
                        break
                        
            elif fact.concept.qname.localName == 'SecurityCodeDEI':
                print("証券コード", fact.value)
                security_code = fact.value

            elif fact.concept.qname.localName == 'FilerNameInJapaneseDEI':
                print("企業名", fact.value)
                filer_name_jp = fact.value
                
            elif fact.concept.qname.localName == 'PriceEarningsRatioSummaryOfBusinessResults':
                price_earningS_ratio = fact.value
                price_earningS_ratios.append(price_earningS_ratio)
            
            elif fact.concept.qname.localName == 'NetAssetsPerShareSummaryOfBusinessResults':
                print("BPS", fact.value)
                net_asset_per_share = fact.value
                net_asset_per_shares.append(net_asset_per_share)
                
            elif fact.concept.qname.localName == 'BasicEarningsLossPerShareSummaryOfBusinessResults':
                print("EPS", fact.value)
                basic_earnings_per_share = fact.value
                basic_earnings_per_shares.append(basic_earnings_per_share)
            
            elif fact.concept.qname.localName == 'DilutedEarningsPerShareSummaryOfBusinessResults':
                print("調整後EPS", fact.value)
                diluted_earnings_per_share = fact.value
                diluted_earnings_per_shares.append(diluted_earnings_per_share)
                
            elif fact.concept.qname.localName == 'DividendPerShareDividendsOfSurplus':
                print("DPS", fact.value)
                dividend_per_share = fact.value
                dividend_per_shares.append(dividend_per_share)
                      
            elif fact.concept.qname.localName == 'RateOfReturnOnEquitySummaryOfBusinessResults':
                print("ROE", fact.value)
                return_on_equity = fact.value
                return_on_equitys.append(return_on_equity)
            
            elif fact.concept.qname.localName == 'PayoutRatioSummaryOfBusinessResults':
                print("配当性向", fact.value)
                dividend_payout_ratio = fact.value
                dividend_payout_ratios.append(dividend_payout_ratio)
                
            elif fact.concept.qname.localName == 'NetCashProvidedByUsedInOperatingActivitiesSummaryOfBusinessResults':
                print("キャッシュフロー（営業）", fact.value)
                cash_flow_from_main = fact.value
                cash_flow_from_mains.append(cash_flow_from_main)
                
            elif fact.concept.qname.localName == 'NetCashProvidedByUsedInInvestingActivitiesSummaryOfBusinessResults':
                print("キャッシュフロー（投資）", fact.value)
                cash_flow_from_investing = fact.value
                cash_flow_from_investings.append(cash_flow_from_investing) 
                
            elif fact.concept.qname.localName == 'IncomeBeforeIncomeTaxes':
                print("当期純利益", fact.value)
                net_income = fact.value
                net_incomes.append(net_income)

        company_info_list.append(edinet_code)
        company_info_list.append(security_code)
        company_info_list.append(filer_name_jp)
        company_info_list.append(industry_code)
        company_info_list.append(price_earningS_ratios)
        company_info_list.append(net_asset_per_shares)
        company_info_list.append(basic_earnings_per_shares)
        company_info_list.append(diluted_earnings_per_shares)

        company_info_list.append(dividend_per_shares)
        company_info_list.append(return_on_equitys)
        company_info_list.append(dividend_payout_ratios)
        company_info_list.append(cash_flow_from_mains)
        company_info_list.append(cash_flow_from_investings)
        company_info_list.append(net_incomes)

        edinet_company_info_list.append(company_info_list)

    return edinet_company_info_list

In [ ]:

##添付した'EdinetcodeDlInfo.csv'というファイルを任意のフォルダにダウンロードする
##ダウンロードした'EdinetcodeDlInfo.csv'までのパスを'edinetcodedlinfo_filepath'という変数に入れる
edinetcodedlinfo_filepath = r'E://★★★★★有価証券報告書//EDINETCODE//EdinetcodeDlInfo.csv'
edinet_info_list = make_edinet_info_list(edinetcodedlinfo_filepath)

##スクリプトファイルである'finance_zip.ipynb'でダウンロードしたzipファイルのパスを指定。'finance_zip.ipynb'で指定したパスと同じでよい
zip_dir = 'E://★★★★★有価証券報告書//デモ//'

##変数'zip_dir'にある変数の末尾に'XBRL//PublicDoc//*.xbrl'を追加するのみでよい
xbrl_file_expressions = 'E://★★★★★有価証券報告書//デモ//XBRL//PublicDoc//*.xbrl'
xbrl_files = unzip_file(zip_dir,xbrl_file_expressions)

##財務諸表の情報が含まれたリストが作成される
edinet_company_info_list = make_edinet_company_info_list(xbrl_files,edinet_info_list)